# Three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ and $1s^2 2s2p\: ^{1,3}P_1$ levels of neutral beryllium

In [1]:
using JAC


In JAC, an restricted-active space (RAS) expansion is to a particular (atomic) representation and, more generally, refers to systematically enlarged SCF computations that are performed stepwise in order to include additional layers of active orbitals. RAS computations are based on one several *reference configurations*, a well-defined *level symmetry* for all CSF bases under consideration as well as on virtual single- (S), double- (D), triple-excitations (T), etc. of electrons from valence orbitals $\to$ valence+virtual orbitals from the (so-called) *active* set. These SD++ excitations can be chosen differently for each step of the RAS computation, similar as the orbitals which are kept *frozen* during a particular step. More often than not, another layer of active orbitals is added and optimized in each step, while all orbitals from previous steps are kept frozen.

In this tutorial, we explain how a three-step RAS expansion for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium as well as for its two lowest-excited $1s^2 2s2p\: ^{1,3}P_1$ levels can be readily generated. For such a representation (of one or several atomic states), we first compute the orbitals for the $1s^2 2s^2$ reference configuration (in some configuration-average approach), and then stepwise enlarge the active orbitals by $2s,\: 2p$ (step$_1$), $3s,\: 3p,\: 3d$ (step$_2$), and finally $4s,\: ...,\: 4f$ (step$_3$). In all these steps, the $1s$ orbitals as well as the orbitals from the previous steps are kept frozen to ensure a proper convergence of the orbitals.

Before we shall explain further details about such RAS expansions (and computations), however, let us however look again at the `struct Atomic.Representation` from which the RAS representation can be *generate*(d) as usual:

In [2]:
? Atomic.Representation

`struct  Atomic.Representation`       ... a struct for defining an atomic representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`Atomic.Representation()`  ... constructor for an 'empty' instance of the a variable::Atomic.Representation


For a RAS expansion, the `repType` must be a `RasExpansion()`, while all virtual excitations of the stepwise enlarged computations refer to the (list of) reference configurations `refConfigs`.

In [3]:
? Atomic.RasExpansion

`struct  Atomic.RasExpansion    <:  AbstractRepresentationType`       ... a struct to represent (and generate) a restricted active-space representation.

```
+ symmetry         ::LevelSymmetry             ... Symmetry of the levels/CSF in the many-electron basis.
+ NoElectrons      ::Int64                     ... Number of electrons.
+ steps            ::Array{Atomic.RasStep,1}   ... List of SCF steps that are to be done in this model computation.
+ settings         ::Atomic.RasSettings        ... Settings for the given RAS computation
```

---

`Atomic.RasExpansion()`  ... constructor for an 'empty' instance of the a variable::Atomic.RasExpansion


In this RAS expansion, we have to specify the (level) `symmetry` of automtically generated CSF expansions as well as *list of* `steps`, which specify the virtual excitations as well as those orbitals which are kept to be frozen during this particular step. These steps are *decribed* by:

In [4]:
? RasStep

search: RasStep RasSettings uppercasefirst lowercasefirst



`struct  Atomic.RasStep`       ... specifies an individual step of a (relativistic) restricted active space computation for a set of levels. This struct          comprises all information to generate the orbital basis and to perform the associated SCF and multiplet computations for a          selected number of levels.

```
+ seFrom            ::Array{Shell,1}        ... Single-excitations from shells   [sh_1, sh_2, ...]
+ seTo              ::Array{Shell,1}        ... Single-excitations to shells  [sh_1, sh_2, ...]
+ deFrom            ::Array{Shell,1}        ... Double-excitations from shells   [sh_1, sh_2, ...]
+ deTo              ::Array{Shell,1}        ... Double-excitations to shells  [sh_1, sh_2, ...]
+ teFrom            ::Array{Shell,1}        ... Triple-excitations from shells   [sh_1, sh_2, ...]
+ teTo              ::Array{Shell,1}        ... Triple-excitations to shells  [sh_1, sh_2, ...]
+ qeFrom            ::Array{Shell,1}        ... Quadrupole-excitations from shells   [sh_1, sh_2, ...]
+ qeTo              ::Array{Shell,1}        ... Quadrupole-excitations to shells  [sh_1, sh_2, ...]
+ frozenShells      ::Array{Shell,1}        ... List of shells that are kept 'frozen' in this step.
+ constraints       ::Array{String,1}       ... List of Strings to define 'constraints/restrictions' to the generated CSF basis.
```

---

`Atomic.RasStep()`  ... constructor for an 'empty' instance of a variable::Atomic.RasStep

---

`Atomic.RasStep(rasStep::Atomic.RasStep;`

```
                seFrom::Array{Shell,1}=Shell[], seTo::Array{Shell,1}=Shell[], 
                deFrom::Array{Shell,1}=Shell[], deTo::Array{Shell,1}=Shell[], 
                teFrom::Array{Shell,1}=Shell[], teTo::Array{Shell,1}=Shell[], 
                qeFrom::Array{Shell,1}=Shell[], qeTo::Array{Shell,1}=Shell[], 
                frozen::Array{Shell,1}=Shell[], constraints::Array{String,1}=String[]  
                
... constructor for modifying the given rasStep by specifying all excitations, frozen shells and constraints optionally.
```


with quite an obvious meaning of single excitations (se), double excitations (de), etc., and where `from` and `to` refer lists of active shells. For each step, we can also specify which `frozenShells` are kept frozen and which additional constraints should be taken into account in generating the CSF lists. **We note, however, that presently no constraints are taken into account in these RAS expansions.** 

Apart from the `steps`, various details about the RAS computations can be specified in terms of the `settings::RasSettings` above, and which apply (equally) to *all* steps:

In [5]:
? RasSettings

search: RasSettings



`struct  Atomic.RasSettings`       ... a struct for defining the settings for a restricted active-space computations.

```
+ levelsScf            ::Array{Int64,1}     ... Levels on which the optimization need to be carried out.
+ maxIterationsScf     ::Int64              ... maximum number of SCF iterations in each RAS step.
+ accuracyScf          ::Float64            ... convergence criterion for the SCF field.

+ breitCI              ::Bool               ... logical flag to include Breit interactions.
+ selectLevelsCI       ::Bool               ... true, if specific level (number)s have been selected.
+ selectedLevelsCI     ::Array{Int64,1}     ... Level number that have been selected.
```

---

`Atomic.RasSettings()`  ... constructor for setting the default values.


These settings enables one to specify the levels on which the SCF is optimized as well as the maximum number of iterations and requested accuracy in these computations. In addition, the flag `breitCI` determines the use of the Breit interaction in the set-up (and diagonalization) of the Hamiltonian matrix; this flag does not affect the SCF of the *next* RAS step but may influence further computations, if the multiplets of the individual steps are subsequently used in computations of properties and processes. We can look for the default settings as well as specify these settings explictly by:

In [6]:
Atomic.RasSettings()

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       false  
selectedLevelsCI:     Int64[]  


In [7]:
rasSettings = RasSettings([1], 24, 1.0e-6, false, true, [1,2,3] )

levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       true  
selectedLevelsCI:     [1, 2, 3]  


The last two entries to these settings tells JAC that we wish to compute (and print) always the lowest three levels of each Hamiltonian matrix; **this feature is not done properly at present since, until now, the whole Hamiltonian matrix is diagonalized (and printed) in all cases.**

We are now prepared to define the RAS expansion as a whole, i.e. a name (mainly for the intermediate and final printout), the reference configuration(s) as well as in turn all (three) steps of a RAS computation for the $1s^2 2s^2\: ^1S_0$ ground level of neutral beryllium:

In [8]:
name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
refConfigs  = [Configuration("[He] 2s^2")]

1-element Array{Configuration,1}:
 Configuration: 1s^2 2s^2 

We also specify that we will keep the $1s$ orbital frozen in the computation of all layers beyond those in the reference configurations as well as that excitations are to be taken from the $2s$ shell:

In [9]:
frozen      = [Shell("1s")]
from        = [Shell("2s")]

1-element Array{Shell,1}:
 2s

In [10]:
to          = [Shell("2s"), Shell("2p")]
step1       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }


Similarly, we can specify the second and third step by using (and overwriting) the information provided for the previous step; the use of deepcopy() ensures that no internal calls are repeated if these cells are called in different order:

In [11]:
append!(frozen, [Shell("2s"), Shell("2p")])
append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }


In [12]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }


With the definition of all *steps*, we can now define the RAS expansion as a whole and subsequently *generate* it:

In [13]:
wa          = Representation(name, Nuclear.Model(4.), Radial.Grid(true), refConfigs, 
                             RasExpansion(LevelSymmetry(0, Basics.plus), 4, [step1, step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 2s^2 ^1S_0 ground state for Z = 4.0 and with reference configurations: 
   1s^2 2s^2 ,  
representation type:   RAS expansion for symmétry 0 + and with 3 steps:
RasStep[
CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1]  
maxIterationsScf:     2

In [14]:
wb = generate(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27214942e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.70620128e-07    
      3      3s_1/2      -8.89087769e-01    -8.89078304e-01    -1.06460554e-05    
      4      4s_1/2      -5.00168012e-01    -5.00086566e-01    -1.62

Dict{String,Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

Of course, analogue computations can be readily carried out also for the (two) $1s^2 2s2p\: ^{1,3}P_1$ excited levels of neutral beryllium, and where we use major parts of the previous computation:

In [15]:
name2        = "Beryllium 1s^2 2s 2p ^1,3P_1 excited levels"
refConfigs2  = [Configuration("[He] 2s 2p")]
wa2          = Representation(name2, Nuclear.Model(4.), Radial.Grid(true), refConfigs2, 
                              RasExpansion(LevelSymmetry(1, Basics.minus), 4, [step1, step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 2s 2p ^1,3P_1 excited levels for Z = 4.0 and with reference configurations: 
   1s^2 2s^1 2p^1 ,  
representation type:   RAS expansion for symmétry 1 - and with 3 steps:
RasStep[
CI or RAS step with 1 (explicitly) frozen shell(s): Shell[1s]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p }
   Doubles from:          { 2s }   ... to { 2s, 2p }
, 
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1]  
maxIterations

In [16]:
wb2 = generate(wa2, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^0 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2s_1/2^1 2p_1/2^1 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27214942e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.70620128e-07    
      3      3s_1/2      -8.89087769e-01    -8.89078304e-01    -1.0646

Dict{String,Any} with 4 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step3"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

All *results* of this and the previous RAS expansion are kept by the (two) dictionaries `wb2` and `wb` above. We can use these variables to extract, for instance, the multiplets as obtained from the first step:

In [17]:
wm = wb["reference multiplet"];   wm2 = wb2["reference multiplet"]

name:        Reference multiplet:  
levels:      Level[Level: J = 0, M = 0, parity = minus, index = 1 
energy:         -14.504262148645418  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 1.0, 0.0]  
, Level: J = 1, M = 1, parity = minus, index = 2 
energy:         -14.504252511938617  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [-0.577291, 0.0, 0.0, 0.816538]  
, Level: J = 2, M = 2, parity = minus, index = 3 
energy:         -14.504233237802351  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 1.0, 0.0, 0.0]  
, Level: J = 1, M = 1, parity = minus, index = 4 
energy:         -14.377639119138909  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.816538, 0.0, 0.0, 0.577291]  
]  


Obviously, these (reference) multiplets describe (three) different *level* since they are built from the $1s^2 2s^2$ and $1s^2 2s 2p$ configurations:

In [18]:
println("Energies from references conf:  $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from references conf:  -14.570469412671915   -14.504262148645418   -14.504252511938617   


We can see how these energies are *lowered* or, at least, modified due to the incorporation of additional correlation layers with each step:

In [19]:
wm = wb["step1"];   wm2 = wb2["step1"]
println("Energies from step 1:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from step 1:           -14.605107808213766   -14.504251330609254   -14.377640158541919   


In [20]:
wm = wb["step2"];   wm2 = wb2["step2"]
println("Energies from step 2:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)   ")

Energies from step 2:           -14.615971325241643   -14.506208864194985   -14.382376531443889   


In [21]:
wm = wb["step3"];   wm2 = wb2["step3"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)   $(wm2.levels[2].energy)  ")

Energies from step 3:           -14.616791191361385   -14.506538099069052   -14.383373126913394  


Finally, we could test how much we benefit by starting from the $1s^2 (2s^2 + 2p^2)$ multi-reference configuration but, otherwise, by performing analogue steps. In this case, of course, we need to allow excitations from $2p$ and can skip step$_2$ since these excitations are now included already into the reference configurations:

In [22]:
frozen       = [Shell("1s"), Shell("2s"), Shell("2p")]
from         = [Shell("2s"), Shell("2p")]

2-element Array{Shell,1}:
 2s
 2p

In [23]:
to          = [Shell("2s"), Shell("2p"), Shell("3s"), Shell("3p"), Shell("3d")]
step2       = RasStep(RasStep(); seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }


In [24]:
append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
step3       = RasStep(step2; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))


CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }


In [25]:
name3        = "Beryllium 1s^2 (2s^2 + 2p^2) ^1S_0 ground state with multi-reference configurations"
refConfigs3  = [Configuration("[He] 2s^2"), Configuration("[He] 2p^2")]
wa3          = Representation(name3, Nuclear.Model(4.), Radial.Grid(true), refConfigs3, 
                              RasExpansion(LevelSymmetry(0, Basics.plus), 4, [step2, step3], rasSettings) )

Atomic representation:   Beryllium 1s^2 (2s^2 + 2p^2) ^1S_0 ground state with multi-reference configurations for Z = 4.0 and with reference configurations: 
   1s^2 2s^2 ,  1s^2 2p^2 ,  
representation type:   RAS expansion for symmétry 0 + and with 2 steps:
RasStep[
CI or RAS step with 3 (explicitly) frozen shell(s): Shell[1s, 2s, 2p]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d }
, 
CI or RAS step with 6 (explicitly) frozen shell(s): Shell[1s, 2s, 2p, 3s, 3p, 3d]  ... and virtual excitations
   Singles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
   Doubles from:          { 2s, 2p }   ... to { 2s, 2p, 3s, 3p, 3d, 4s, 4p, 4d, 4f }
]  
... and the current settings:
levelsScf:            [1]  
maxIterationsScf:     24  
accuracyScf:          1.0e-6  
breitCI:              false  
selectLevelsCI:       true  
selectedLevelsCI:     [1, 2, 3] 

In [26]:
wb3 = generate(wa3, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 
>> include Configuration: 1s_1/2^2 2p_1/2^0 2p_3/2^2 
>> include Configuration: 1s_1/2^2 2p_1/2^1 2p_3/2^1 
>> include Configuration: 1s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 4.0 with mass = 8.08, radius R = 2.247554858082373 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -8.00170451e+00    -8.00170477e+00    +3.27214942e-08    
      2      2s_1/2      -2.00053329e+00    -2.00053275e+00    -2.706201

Dict{String,Any} with 3 entries:
  "reference multiplet" => name:        Reference multiplet:  …
  "step2"               => name:        Multiplet:  …
  "step1"               => name:        Multiplet:  …

Again, we can print the final energy and compare the results with the single-reference configurations:

In [27]:
wm = wb["step3"];   wm3 = wb3["step2"]
println("Energies from step 3:           $(wm.levels[1].energy)   $(wm2.levels[1].energy)  ")

Energies from step 3:           -14.616791191361385   -14.506538099069052  
